# Neo4j

## Conexión con Cassandra y borrado de datos

In [1]:
%load_ext cypher

In [2]:
from py2neo import Graph, Node, Relationship

graph = Graph()

In [3]:
# Borrado de todos los nodos y relaciones
graph.delete_all()

## Carga de datos en Pandas

A partir del fichero Excel desnormalizamos los datos

In [5]:
import pandas as pd
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   minuto              7624 non-null   int64         
 3   hora                7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6882 non-null   object        
 6   actividad_completa  7621 non-null   object        
 7   actividad           7621 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6086 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


## Carga de datos en Neo4j

Limpiamos los datos para que no tengan nulos y evitar problemas con la sentencia MERGE de Neo

In [7]:
df.loc[pd.isnull(df['comercio']),['comercio']] = 'Desconocido'
df.loc[pd.isnull(df['organizacion']),['organizacion']] = 'Desconocido'
df.loc[pd.isnull(df['actividad']),['actividad']] = 'Desconocido'
df.loc[pd.isnull(df['actividad_completa']),['actividad_completa']] = 'Desconocido'

La carga se realizará en Neo a partir de la lista de datos en formato JSON, generados con Pandas

In [8]:
import json

json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

En Neo se van a crear 3 tipos de nodos y 2 tipos de relaciones

<br><br> 

<img src="images/Modelo%20Neo4j.png" width=600 height=500>

<br><br> 

Adicionalmente se van a guardar los siguientes atributos:

**Nodo Comercio**
- Nombre del comercio
- Actividad a la que se dedica
- Categoría de actividad (Podríamos haber creado un tipo de nodo independente)

**Nodo Persona**
- Nombre de la persona

**Nodo Organización**
- Nombre de la organización

**Nodo Movimiento**
- Fecha en formato UNIX Timestamp (el mismo formato que está en el JSON)
- hora y minuto de la compra
- Importe de la compra

Respecto a las relaciones, vamos a almacenar los siguientes atributos:

**Relación PERTECENE**
- Función que hace la persona en una organización

Creamos los nodos y relaciones en Neo4j simplemente iterando sobre la lista de JSONs.

In [10]:
import datetime

for index, movimiento_json in enumerate(json_list):
    # Nodos
    persona = Node("Persona", nombre=movimiento_json["nombre"])
    comercio = Node("Comercio", nombre=movimiento_json["comercio"], 
                                actividad = movimiento_json["actividad"],
                                actividad_completa = movimiento_json["actividad_completa"])
    organizacion = Node("Organizacion", nombre = movimiento_json["organizacion"])
    movimiento = Node("Movimiento",
            fecha = movimiento_json['fecha'],
            hora =  movimiento_json["hora"],
            minuto =  movimiento_json["minuto"],
            importe = movimiento_json["importe"])
    
        
    # Relaciones  
    rel_persona_organizacion = Relationship(persona, "PERTENECE", organizacion,
                                           funcion = movimiento_json["funcion"])
    rel_persona_movimiento = Relationship(persona, "REALIZA", movimiento)
    rel_movimiento_comercio = Relationship(movimiento, "OCURRE", comercio)
       
    graph.merge(persona | comercio | organizacion | rel_persona_organizacion)
    graph.create(movimiento | rel_persona_movimiento | rel_movimiento_comercio)
    
    if index % 500 == 0:
        print(index)


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


<br><br> 

<img src="images/Resultado%20Neo4j.png" width=800 height=600>

<br><br> 

### Comprobaciones previas ...

Número de movimientos ...

In [11]:
%%cypher
MATCH (:Movimiento)
RETURN count(*)

1 rows affected.


count(*)
7624


Número de personas ...

In [12]:
%%cypher
MATCH (:Persona)
RETURN count(*)

1 rows affected.


count(*)
82


Número de comercios

In [13]:
%%cypher
MATCH (c:Comercio)
WHERE c.nombre <> 'Desconocido'
RETURN count(*)

1 rows affected.


count(*)
3177


Número de organizaciones

In [14]:
%%cypher
MATCH (o:Organizacion)
WHERE o.nombre <> 'Desconocido'
RETURN count(*)

1 rows affected.


count(*)
10


All good!

## Funciones de utilidad para la consulta de datos

Neo4j, en su versión actual, no puede guardar datos de tipo fecha en los atributos, por lo que se almacena en formato TIMESTAMP de Unix (el formato original).  

La siguiente función de utilidad convierte una columna de una DataFrame en formato TIMESTAMP a Fecha

In [15]:
def transform_date(df, column_name):
    if column_name in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df[column_name] = pd.to_datetime(df[column_name], unit = 'ms')
        
    return df

Función de utilidad para convertir los resultados devueltos por la base de datos en un DataFrame de Pandas

In [16]:
def get_dataframe(data):
    df = pd.DataFrame(list(data), columns = data.keys())
        
    return df

## Consulta a Neo4j

### Los 10 movimientos mas caros

Para obtener las lista de personas que mas han gastado simplemente relacionamos los 3 tipos de nodos y devolvemos los campos que nos interesan

In [17]:
%%cypher
MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
RETURN persona.nombre, comercio.actividad_completa, movimiento.importe
ORDER BY movimiento.importe DESC
LIMIT 10

10 rows affected.


persona.nombre,comercio.actividad_completa,movimiento.importe
Carmen Contreras Gómez,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",9076.76
Ildefonso José Sánchez Barcoj,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),7500.0
Ildefonso José Sánchez Barcoj,EL CORTE INGLES,6593.2
Miguel Blesa de la Parra,EL CORTE INGLES,6397.31
Ramón Ferraz Ricarte,EL CORTE INGLES,6248.0
Ramón Ferraz Ricarte,HYATT HOTELS,6160.81
Carmen Contreras Gómez,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",6102.68
Mariano Pérez Claver,AGENCIAS DE VIAJES,6000.0
María Carmen Cafranga Cavestany,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.0
Estanislao Rodríguez-Ponga Salamanca,EL CORTE INGLES,5500.0


Utilizamos la función ** graph.run()** para obtener un ResultSet de datos, que es transformada en un Dataframe de Pandas

In [18]:
query = """
MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
RETURN persona.nombre, comercio.actividad_completa, movimiento.importe
ORDER BY movimiento.importe DESC
LIMIT 10
""" 

data = graph.run(query)
df = get_dataframe(data)
df = transform_date(df, 'compra.fecha')
df

,persona.nombre,comercio.actividad_completa,movimiento.importe
0,Carmen Contreras Gómez,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",9076.76
1,Ildefonso José Sánchez Barcoj,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),7500.00
2,Ildefonso José Sánchez Barcoj,EL CORTE INGLES,6593.20
3,Miguel Blesa de la Parra,EL CORTE INGLES,6397.31
4,Ramón Ferraz Ricarte,EL CORTE INGLES,6248.00
5,Ramón Ferraz Ricarte,HYATT HOTELS,6160.81
6,Carmen Contreras Gómez,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",6102.68
7,Mariano Pérez Claver,AGENCIAS DE VIAJES,6000.00
8,María Carmen Cafranga Cavestany,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.00
9,Estanislao Rodríguez-Ponga Salamanca,EL CORTE INGLES,5500.00


La misma query que el caso anterior pero utilizando un método distinto.  
El fin es el mismo: Una DataFrame de Pandas

In [19]:
df = %cypher MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio) \
             RETURN persona.nombre, comercio.actividad_completa, movimiento.importe \
             ORDER BY movimiento.importe DESC \
             LIMIT 10
            
df.get_dataframe()

10 rows affected.


,persona.nombre,comercio.actividad_completa,movimiento.importe
0,Carmen Contreras Gómez,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",9076.76
1,Ildefonso José Sánchez Barcoj,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),7500.00
2,Ildefonso José Sánchez Barcoj,EL CORTE INGLES,6593.20
3,Miguel Blesa de la Parra,EL CORTE INGLES,6397.31
4,Ramón Ferraz Ricarte,EL CORTE INGLES,6248.00
5,Ramón Ferraz Ricarte,HYATT HOTELS,6160.81
6,Carmen Contreras Gómez,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",6102.68
7,Mariano Pérez Claver,AGENCIAS DE VIAJES,6000.00
8,María Carmen Cafranga Cavestany,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.00
9,Estanislao Rodríguez-Ponga Salamanca,EL CORTE INGLES,5500.00


### Importes de una persona agrupados por actividad

En este caso se realiza una agrupación de los datos. Observa que Neo4j lo realiza automáticamente al utilizar una función de agrupación de datos como **SUM()**

In [20]:
%%cypher
MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
WHERE comercio.actividad = 'HOGAR'
RETURN persona.nombre, comercio.actividad_completa, SUM(movimiento.importe) as importe
ORDER BY importe DESC
LIMIT 10


10 rows affected.


persona.nombre,comercio.actividad_completa,importe
Mariano Pérez Claver,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",4321.23
Carlos María Martínez Martínez,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2127.6
Javier de Miguel Sánchez,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",1588.06
Francisco Baquero Noriega,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1371.0
Rafael Spottorno Díaz Caro,MIRO ESTABLECIMIENTOS,1198.0
Jesús Pedroche Nieto,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1190.19
Maria Mercedes de la Merced Monge,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1063.85
Mariano Pérez Claver,FLORES Y PLANTAS,1054.23
Carlos Vela García,FLORES Y PLANTAS,1042.3899999999999
Pedro Bugidos Garay,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1000.0


In [21]:
query = """
MATCH (organizacion:Organizacion)-[:PERTENECE]-(persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
WHERE persona.nombre = {name}
RETURN persona.nombre, comercio.actividad, SUM(movimiento.importe) as importe
ORDER BY importe DESC
"""

data = graph.run(query, name='Mariano Pérez Claver')
get_dataframe(data)

,persona.nombre,comercio.actividad,importe
0,Mariano Pérez Claver,CA$H,10222.17
1,Mariano Pérez Claver,HOGAR,5548.23
2,Mariano Pérez Claver,SALUD,3749.50
3,Mariano Pérez Claver,RESTAURANTE,2659.54
4,Mariano Pérez Claver,SUPERMERCADO,2231.46
5,Mariano Pérez Claver,COCHE,2124.28
6,Mariano Pérez Claver,COMPRA BIENES,1921.21
7,Mariano Pérez Claver,BARCO,1909.69
8,Mariano Pérez Claver,HOTEL,1491.21
9,Mariano Pérez Claver,TREN,1042.49
